# 3. Quantify extracellular RNA

After defining extracellular transcripts (previous notebooks), we will now focus on characterizing and quantifying the presence of exRNA for the different genes profiled in the dataset employing different strategies and tests

## Import packages

In [3]:
import spatialdata as sd
import troutpy 

## Read the SpatialData object

We read the processed Spatialdata object, which was previously stored as .zarr

In [4]:
xenium_path_cropped='/media/sergio/Discovair_final/mousebrain_prime_crop_points2regions_annotated.zarr'
sdata=sd.read_zarr(xenium_path_cropped)

/home/sergio/.local/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/sergio/.local/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/sergio/.local/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/sergio/.local/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/sergio/.local/lib/python3.10/site-packages/zarr/creation.py:614: UserWarning: ignoring keyword argument 'read_only'
  compressor, fill_value = _kwargs_compat(compressor, fill_value, kwargs)
/home/sergio/.local/

## Testing for expression over noise levels

We implemented `troutpy.tl.quantify_overexpression` to identify **overexpressed genes relative to a noise threshold**.

Essentially, it computes a threshold based on the counts of specified control features and compares gene counts against this threshold to determine overexpression. The function calculates log-fold changes for each gene, annotates metadata with these results. It returns updated spatial data along with per-gene scores and the calculated threshold

In [5]:
control_codewords = ['negative_control_probe','unassigned_codeword', 'deprecated_codeword', 
'genomic_control_probe', 'negative_control_codeword']

troutpy.tl.quantify_overexpression(sdata,layer='transcripts',codeword_column="codeword_category",
control_codewords=control_codewords,gene_id_column="feature_name",percentile_threshold=99.99)

/home/sergio/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


Added 'xrna_metadata' table with 13035 unique genes to the SpatialData object.


/home/sergio/.local/lib/python3.10/site-packages/troutpy/tl/quantify_xrna.py:216: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  sdata['xrna_metadata'].var['control_probe']=sdata['xrna_metadata'].var['control_probe'].fillna(False)


## Testing for proportion of extracellular transcripts

The `troutpy.tl.extracellular_enrichment` function computes the proportions of extracellular and intracellular transcripts for each gene. 

Essentially, the function calculates: (1) the proportion of transcripts of each gene present extracellularly and (2) the log fold change of extracellular to intracellular proportions.  These results into the `sdata` object under the `xrna_metadata` layer.

In [6]:
troutpy.tl.extracellular_enrichment(sdata)

/home/sergio/.local/lib/python3.10/site-packages/pandas/core/arraylike.py:399: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)


## Testing for Spatial Variability

The `troutpy.tl.spatial_variability` function quantifies the spatial variability of extracellular RNA using Moran's I, a metric for spatial autocorrelation.

In [9]:
troutpy.tl.spatial_variability(sdata,  gene_id_key='feature_name',coords_keys=['x','y'], n_neighbors=10,binsize=20)

Extracting gene counts: 100%|██████████| 13031/13031 [00:00<00:00, 16970.23it/s]


## Testing for local density

The `spatial_colocalization` function evaluates the spatial colocalization of extracellular RNA transcripts in spatial transcriptomics data by identifying regions where transcripts from the same or different genes are found in close proximity. The function calculates the proportion of colocalized transcripts for each gene, using a user-defined threshold to define colocalization, and integrates these results into the metadata of the dataset for downstream analysis.

The function calculates the proportion of colocalized transcripts for each gene based on a user-defined threshold, leveraging kernel density estimation to compute spatial gene expression densities. Results are integrated into the `sdata` object under the `xrna_metadata` layer.

In [ ]:
troutpy.tl.spatial_colocalization(sdata, coords_keys=['x', 'y'], gene_id_key='feature_name',resolution=1000,binsize=5, threshold_colocalized=1 ,copy=False)

Extracting gene counts: 100%|██████████| 13031/13031 [00:07<00:00, 1787.40it/s]


## Save SpatialData as zarr

The resulting dataset is finally stored in a new zarr object

In [8]:
xenium_path_cropped='/media/sergio/Discovair_final/mousebrain_prime_crop_quantified.zarr'
sdata.write(xenium_path_cropped,overwrite=True)

INFO     The SpatialData object is not self-contained (i.e. it contains some elements that are Dask-backed from    
         locations outside /media/sergio/Discovair_final/mousebrain_prime_crop_quantified.zarr). Please see the    
         documentation of `is_self_contained()` to understand the implications of working with SpatialData objects 
         that are not self-contained.                                                                              
INFO     The Zarr backing store has been changed from                                                              
         /media/sergio/Discovair_final/mousebrain_prime_crop_points2regions_annotated.zarr the new file path:      
         /media/sergio/Discovair_final/mousebrain_prime_crop_quantified.zarr                                       


### ---TO DO--- Create visualization for the output of different tests

In [38]:
exrna_metrics=sdata['xrna_metadata'].var
exrna_metrics=exrna_metrics[exrna_metrics['count']>120]
exrna_metrics_filt=exrna_metrics.loc[:,['logfoldratio_over_noise','logfoldratio_extracellular','moran_I','proportion_of_colocalized']]